<center>
  <h1 style="background-color: #80dfffff; color: #137a91ff; padding: 10px">
    <strong>Deployment</strong>
  </h1>
</center>

In [ ]:
# Importing necessary libraries
import joblib
import pandas as pd
import numpy as np

from Classes import Categorical_Correction, Outlier_Treatment, Missing_Value_Treatment, Typecasting, Feature_Engineering, Encoder, Scaler, Feature_Selection

from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from sklearn.neighbors import KNeighborsRegressor

from sklearn.compose import TransformedTargetRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

random_state = 42

In [ ]:
# Load train and test data and split into features and target
df_train = pd.read_csv("train.csv")  
X_train = df_train.drop(columns=["price"])  
y_train = df_train["price"] 
df_test = pd.read_csv("test.csv") 

df_train.set_index('carID', inplace=True)
df_test.set_index('carID', inplace=True)

In [ ]:
df_test.drop ('paintQuality%', axis =1 , inplace =True)

# Pre processing steps not included in the pipeline
df_test.loc[df_test['year']>2020, 'year'] = np.nan
df_test.loc[df_test['mileage']<0, 'mileage'] = np.nan
df_test.loc[df_test['tax']<0,'tax'] = np.nan
df_test.loc[df_test['mpg']<=0, 'mpg'] = np.nan
df_test.loc[df_test['previousOwners']< 0, 'previousOwners'] = np.nan
df_test.loc[df_test['engineSize']<= 0, 'engineSize'] = np.nan
df_test.loc[df_test['mpg'] < 8, 'mpg'] = np.nan
df_test.loc[df_test['engineSize'] < 1, 'engineSize'] = np.nan


df_test['year'] = np.floor(df_test['year'])
df_test['previousOwners'] = np.floor(df_test['previousOwners'])

for feat in ['mileage', 'tax', 'mpg', 'engineSize', 'paintQuality%']:
    df_test[feat] = df_test[feat].round(2)


df_test['Brand']  = df_test['Brand'].where(df_test['Brand'].isna(), df_test['Brand'].astype(str).str.strip().str.upper())
df_test['model']  = df_test['model'].where(df_test['model'].isna(), df_test['model'].astype(str).str.strip().str.upper())
df_test['fuelType']  = df_test['fuelType'].where(df_test['fuelType'].isna(), df_test['fuelType'].astype(str).str.strip().str.upper())
df_test['transmission']  = df_test['transmission'].where(df_test['transmission'].isna(), df_test['transmission'].astype(str).str.strip().str.upper())


df_test.loc[df_test['Brand'] == 'W', 'Brand'] = 'VW'

In [ ]:
# Load the pre-trained model
model = joblib.load("best_model.pkl")

# Fit the model and make predictions
model.fit(X_train, y_train)
predictions = model.predict(df_test)
predictions = predictions.loc[:, "price"]
predictions

In [ ]:
# Save predictions to a CSV file, so we can submit them to Kaggle
predictions.to_csv("predictions.csv", index=True)

In [ ]:
# Save the final model fitted on the entire training data to use for the streamlit interface
joblib.dump(model, "final_model.pkl")
